# 2- Vacation Search - Creating a Customer Travel Destinations Map

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np
import random
import time
import config
import sys

from datetime import datetime
from citipy import citipy 


In [2]:
# Import API key

# g_key = "g_key goes here"

#from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)



In [4]:
# 1. Import the WeatherPy_database.csv file. 
# city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv") # not finding the file
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp(C),Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Cape Town,ZA,-33.9258,18.4232,22.23,27,0,5.91,clear sky,2021-06-06 22:56:22
1,1,Yellowknife,CA,62.4560,-114.3525,10.31,53,100,1.79,overcast clouds,2021-06-06 23:00:20
2,2,Beatrice,US,40.2681,-96.7470,29.36,57,40,3.60,scattered clouds,2021-06-06 23:03:47
3,3,Nome,US,64.5011,-165.4064,2.87,83,100,2.14,overcast clouds,2021-06-06 23:03:49
4,4,Georgetown,MY,5.4112,100.3354,26.96,90,20,1.32,few clouds,2021-06-06 23:00:14
5,5,Ullapool,GB,57.8987,-5.1604,12.19,81,99,0.89,overcast clouds,2021-06-06 23:03:51
6,6,Rikitea,PF,-23.1203,-134.9692,24.87,83,98,6.81,overcast clouds,2021-06-06 22:57:05
7,7,Ushuaia,AR,-54.8000,-68.3000,1.82,95,86,3.91,overcast clouds,2021-06-06 23:00:10
8,8,Busselton,AU,-33.6500,115.3333,7.83,78,0,3.50,clear sky,2021-06-06 23:01:46
9,9,Te Anau,NZ,-45.4167,167.7167,5.84,84,12,0.54,few clouds,2021-06-06 23:03:57


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is your preferable minimum temperature for your trip in degrees centigrade?"))
max_temp = float(input("What is your preferable maximum temperature for your trip in degrees centigrade?"))

What is your preferable minimum temperature for your trip in degrees centigrade?23
What is your preferable maximum temperature for your trip in degrees centigrade?32


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp(C)"] >= min_temp) & (city_data_df["Max Temp(C)"] <= max_temp)]
     
preferred_cities_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp(C),Humidity,Cloudiness,Wind Speed,Current Description,Date
2,2,Beatrice,US,40.2681,-96.7470,29.36,57,40,3.60,scattered clouds,2021-06-06 23:03:47
4,4,Georgetown,MY,5.4112,100.3354,26.96,90,20,1.32,few clouds,2021-06-06 23:00:14
6,6,Rikitea,PF,-23.1203,-134.9692,24.87,83,98,6.81,overcast clouds,2021-06-06 22:57:05
10,10,Sibolga,ID,1.7427,98.7792,23.77,79,76,1.40,broken clouds,2021-06-06 23:03:58
14,14,Arraial Do Cabo,BR,-22.9661,-42.0278,23.34,86,100,9.15,overcast clouds,2021-06-06 23:04:05


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp(C)            0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Date                   0
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp(C),Humidity,Cloudiness,Wind Speed,Current Description,Date
2,2,Beatrice,US,40.2681,-96.7470,29.36,57,40,3.60,scattered clouds,2021-06-06 23:03:47
4,4,Georgetown,MY,5.4112,100.3354,26.96,90,20,1.32,few clouds,2021-06-06 23:00:14
6,6,Rikitea,PF,-23.1203,-134.9692,24.87,83,98,6.81,overcast clouds,2021-06-06 22:57:05
10,10,Sibolga,ID,1.7427,98.7792,23.77,79,76,1.40,broken clouds,2021-06-06 23:03:58
14,14,Arraial Do Cabo,BR,-22.9661,-42.0278,23.34,86,100,9.15,overcast clouds,2021-06-06 23:04:05


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp(C)", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp(C),Current Description,Lat,Lng,Hotel Name
2,Beatrice,US,29.36,scattered clouds,40.2681,-96.7470,
4,Georgetown,MY,26.96,few clouds,5.4112,100.3354,
6,Rikitea,PF,24.87,overcast clouds,-23.1203,-134.9692,
10,Sibolga,ID,23.77,broken clouds,1.7427,98.7792,
14,Arraial Do Cabo,BR,23.34,overcast clouds,-22.9661,-42.0278,
16,Tamandare,BR,24.52,scattered clouds,-8.7597,-35.1047,
17,Albany,US,29.86,scattered clouds,42.6001,-73.9662,
19,Kapaa,US,28.44,few clouds,22.0752,-159.3190,
22,Atuona,PF,26.77,broken clouds,-9.8000,-139.0333,
30,Muncar,ID,23.94,scattered clouds,-8.4333,114.3333,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        
    try:    
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found ....skipping.")    
        

Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df["Hotel Name"].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
clean_hotel_df = hotel_df
clean_hotel_df.head()


,City,Country,Max Temp(C),Current Description,Lat,Lng,Hotel Name
2,Beatrice,US,29.36,scattered clouds,40.2681,-96.7470,Victorian Inn Beatrice
4,Georgetown,MY,26.96,few clouds,5.4112,100.3354,Cititel Penang
6,Rikitea,PF,24.87,overcast clouds,-23.1203,-134.9692,Pension Maro'i
10,Sibolga,ID,23.77,broken clouds,1.7427,98.7792,Hotel Syariah CN Darussalam
14,Arraial Do Cabo,BR,23.34,overcast clouds,-22.9661,-42.0278,Pousada Porto Praia


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "Weatherpy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp(C)} °C</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
#max_temp = hotel_df["Max Temp(C)"]
fig = gmaps.figure(center=(30.0, -80), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))